In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, average_precision_score
import optuna

In [2]:
train_df = pd.read_csv('../../../dataset/open_world/openworld_train.csv')
test_df = pd.read_csv('../../../dataset/open_world/openworld_test.csv')

# Feature / Target Split
X = train_df.drop(columns=["label"]).values
y = train_df["label"].values

X_test = test_df.drop(columns=["label"]).values
y_test = test_df["label"].values

# Train / Validation Split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [3]:
def objective(trial):
    # Search space
    hidden_layer_1 = trial.suggest_int("hidden_layer_1", 128, 1024, step=64)
    hidden_layer_2 = trial.suggest_int("hidden_layer_2", 64, 512, step=32)
    alpha = trial.suggest_loguniform("alpha", 1e-6, 1e-2)
    learning_rate_init = trial.suggest_loguniform("learning_rate_init", 1e-5, 5e-3)
    batch_size = trial.suggest_categorical("batch_size", [64, 128, 256])
    activation = trial.suggest_categorical("activation", ["relu", "tanh"])
    solver = trial.suggest_categorical("solver", ["adam", "sgd"])

    # Define model
    mlp = MLPClassifier(
        hidden_layer_sizes=(hidden_layer_1, hidden_layer_2),
        activation=activation,
        solver=solver,
        alpha=alpha,
        batch_size=batch_size,
        learning_rate='adaptive',
        learning_rate_init=learning_rate_init,
        max_iter=300,
        early_stopping=True,
        n_iter_no_change=15,
        validation_fraction=0.15,
        random_state=42,
        verbose=False
    )

    # Train
    mlp.fit(X_train, y_train)

    # Validation performance
    y_pred = mlp.predict(X_val)
    f1_macro = f1_score(y_val, y_pred, average='macro')
    return f1_macro

In [4]:
# Optuna Study
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=42)
)
study.optimize(objective, n_trials=150, show_progress_bar=True)

print("\n========== OPTUNA BEST RESULT ==========")
print(f"Best Macro-F1: {study.best_value:.4f}")
print("parameters:", study.best_params)

[I 2025-11-21 15:32:48,543] A new study created in memory with name: no-name-f9621ca7-a6db-4ef4-9a75-974b59a330e7


  0%|          | 0/150 [00:00<?, ?it/s]

/var/folders/4q/3301trc50fz6_vl0qx1b71r00000gn/T/ipykernel_36072/3735073601.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform("alpha", 1e-6, 1e-2)
/var/folders/4q/3301trc50fz6_vl0qx1b71r00000gn/T/ipykernel_36072/3735073601.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate_init = trial.suggest_loguniform("learning_rate_init", 1e-5, 5e-3)


[I 2025-11-21 15:34:00,183] Trial 0 finished with value: 0.6134562481390978 and parameters: {'hidden_layer_1': 448, 'hidden_layer_2': 512, 'alpha': 0.0008471801418819979, 'learning_rate_init': 0.00041282053438262235, 'batch_size': 64, 'activation': 'relu', 'solver': 'adam'}. Best is trial 0 with value: 0.6134562481390978.


/var/folders/4q/3301trc50fz6_vl0qx1b71r00000gn/T/ipykernel_36072/3735073601.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform("alpha", 1e-6, 1e-2)
/var/folders/4q/3301trc50fz6_vl0qx1b71r00000gn/T/ipykernel_36072/3735073601.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate_init = trial.suggest_loguniform("learning_rate_init", 1e-5, 5e-3)
/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/var/folders/4q/3301trc50fz6_vl0qx1b71r000

...

[I 2025-11-21 20:00:27,724] Trial 146 finished with value: 0.6460452681625058 and parameters: {'hidden_layer_1': 384, 'hidden_layer_2': 480, 'alpha': 1.0282458899994136e-06, 'learning_rate_init': 0.0007311988877580698, 'batch_size': 64, 'activation': 'tanh', 'solver': 'adam'}. Best is trial 44 with value: 0.667698580452547.


/var/folders/4q/3301trc50fz6_vl0qx1b71r00000gn/T/ipykernel_36072/3735073601.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform("alpha", 1e-6, 1e-2)
/var/folders/4q/3301trc50fz6_vl0qx1b71r00000gn/T/ipykernel_36072/3735073601.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate_init = trial.suggest_loguniform("learning_rate_init", 1e-5, 5e-3)


[I 2025-11-21 20:02:16,894] Trial 147 finished with value: 0.6554203673370271 and parameters: {'hidden_layer_1': 576, 'hidden_layer_2': 512, 'alpha': 2.1262346276391147e-06, 'learning_rate_init': 0.0010423194633431508, 'batch_size': 64, 'activation': 'tanh', 'solver': 'adam'}. Best is trial 44 with value: 0.667698580452547.


/var/folders/4q/3301trc50fz6_vl0qx1b71r00000gn/T/ipykernel_36072/3735073601.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform("alpha", 1e-6, 1e-2)
/var/folders/4q/3301trc50fz6_vl0qx1b71r00000gn/T/ipykernel_36072/3735073601.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate_init = trial.suggest_loguniform("learning_rate_init", 1e-5, 5e-3)
/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/var/folders/4q/3301trc50fz6_vl0qx1b71r000

[I 2025-11-21 20:06:09,958] Trial 148 finished with value: 0.4577785606464843 and parameters: {'hidden_layer_1': 512, 'hidden_layer_2': 512, 'alpha': 1.2340497302721044e-06, 'learning_rate_init': 0.0008286240702189782, 'batch_size': 128, 'activation': 'tanh', 'solver': 'sgd'}. Best is trial 44 with value: 0.667698580452547.
[I 2025-11-21 20:06:42,495] Trial 149 finished with value: 0.6289337033955753 and parameters: {'hidden_layer_1': 256, 'hidden_layer_2': 288, 'alpha': 3.1305622011066945e-06, 'learning_rate_init': 0.0014256987620001435, 'batch_size': 64, 'activation': 'tanh', 'solver': 'adam'}. Best is trial 44 with value: 0.667698580452547.

========== OPTUNA BEST RESULT ==========
Best Macro-F1: 0.6677
parameters: {'hidden_layer_1': 384, 'hidden_layer_2': 512, 'alpha': 1.9155906182703047e-06, 'learning_rate_init': 0.0009118650899147087, 'batch_size': 64, 'activation': 'tanh', 'solver': 'adam'}


In [7]:
# Retrain w/ Best params
best_params = study.best_params
best_model = MLPClassifier(
    hidden_layer_sizes=(best_params["hidden_layer_1"], best_params["hidden_layer_2"]),
    activation=best_params["activation"],
    solver=best_params["solver"],
    alpha=best_params["alpha"],
    batch_size=best_params["batch_size"],
    learning_rate='adaptive',
    learning_rate_init=best_params["learning_rate_init"],
    max_iter=400,
    early_stopping=True,
    n_iter_no_change=15,
    validation_fraction=0.15,
    random_state=42,
    verbose=True
)

print("\n[INFO] Training final model with best params...")
best_model.fit(X_train, y_train)


[INFO] Training final model with best params...
Iteration 1, loss = 2.86273885
Validation score: 0.383005
Iteration 2, loss = 2.37586234
Validation score: 0.426519
Iteration 3, loss = 2.13979097
Validation score: 0.464696
Iteration 4, loss = 1.95503616
Validation score: 0.491379
Iteration 5, loss = 1.78341084
Validation score: 0.525862
Iteration 6, loss = 1.63375976
Validation score: 0.547619
Iteration 7, loss = 1.49842910
Validation score: 0.556650
Iteration 8, loss = 1.37884310
Validation score: 0.569376
Iteration 9, loss = 1.27729898
Validation score: 0.594417
Iteration 10, loss = 1.18715710
Validation score: 0.599343
Iteration 11, loss = 1.11325013
Validation score: 0.605090
Iteration 12, loss = 1.04027024
Validation score: 0.635057
Iteration 13, loss = 0.96878574
Validation score: 0.627668
Iteration 14, loss = 0.91658545
Validation score: 0.638752
Iteration 15, loss = 0.85928099
Validation score: 0.637931
Iteration 16, loss = 0.81078983
Validation score: 0.633005
Iteration 17, lo

MLPClassifier(activation='tanh', alpha=1.9155906182703047e-06, batch_size=64,
              early_stopping=True, hidden_layer_sizes=(384, 512),
              learning_rate='adaptive',
              learning_rate_init=0.0009118650899147087, max_iter=400,
              n_iter_no_change=15, random_state=42, validation_fraction=0.15,
              verbose=True)

In [10]:
test_pred = best_model.predict(X_test)
test_proba = best_model.predict_proba(X_test)

# Accuracy / F1
test_acc = accuracy_score(y_test, test_pred)
test_f1_macro = f1_score(y_test, test_pred, average='macro')
test_f1_micro = f1_score(y_test, test_pred, average='micro')
test_f1_weighted = f1_score(y_test, test_pred, average='weighted')

# ROC-AUC (multi-class, one-vs-rest 방식)
test_roc_auc = roc_auc_score(
    y_test, test_proba,
    multi_class='ovr',
    average='macro'
)

# PR-AUC (multi-class macro)
test_pr_auc = average_precision_score(
    y_test, test_proba,
    average='macro'
)

print("\n========== FINAL TEST RESULTS ==========")
print(f"Accuracy        : {test_acc:.4f}")
print(f"F1 (macro)      : {test_f1_macro:.4f}")
print(f"F1 (micro)      : {test_f1_micro:.4f}")
print(f"F1 (weighted)   : {test_f1_weighted:.4f}")
print(f"ROC-AUC (macro) : {test_roc_auc:.4f}")
print(f"PR-AUC (macro)  : {test_pr_auc:.4f}")


========== FINAL TEST RESULTS ==========
Accuracy        : 0.7234
F1 (macro)      : 0.6584
F1 (micro)      : 0.7234
F1 (weighted)   : 0.7192
ROC-AUC (macro) : 0.9837
PR-AUC (macro)  : 0.7075
